In [2]:
import pandas as pd

def read_votaciones_2014_1v():
    votaciones_2014 = pd.read_excel('./in/MMV_NACIONAL_PRESIDENTE_2014_1v.xlsx')
    return votaciones_2014

# Get the data
votaciones_2014 = read_votaciones_2014_1v()


In [3]:
votaciones_2014.head()

,COD_DPTO,DEPARTAMENTO,COD_MCPIO,MUNICIPIO,ZONA,COD_PUESTO,MESA,COD_CIRCUNS,COD_PARTIDO,PARTIDO,COD_CANDIDATO,CANDIDATO,VOTOS
0,1,ANTIOQUIA,1,MEDELLIN,1,1,1,1,1,Polo democrático,1,LOPEZ,9
1,1,ANTIOQUIA,1,MEDELLIN,1,1,1,1,2,Partido Conservador,2,RAMIREZ,15
2,1,ANTIOQUIA,1,MEDELLIN,1,1,1,1,3,Unidad Nacional,3,SANTOS,37
3,1,ANTIOQUIA,1,MEDELLIN,1,1,1,1,4,Alianza Verde,4,PEÑALOSA,2
4,1,ANTIOQUIA,1,MEDELLIN,1,1,1,1,5,Centro democrático,5,ZULUAGA,42


In [4]:
votaciones_nacional_2014 = votaciones_2014.drop(['COD_DPTO',	'DEPARTAMENTO',	'COD_MCPIO',	'MUNICIPIO',	'ZONA',	'COD_PUESTO',	'MESA',	'COD_CIRCUNS',	'COD_PARTIDO',	'PARTIDO',	'COD_CANDIDATO'], axis = 1)
votaciones_nacional_2014.describe()

,VOTOS
count,701252.000000
mean,18.832766
std,23.064220
min,0.000000
25%,3.000000
50%,10.000000
75%,27.000000
max,610.000000


In [5]:
# Eliminar filas donde CANDIDATO sea alguno de estos valores
valores_a_eliminar = ["VOTOS EN BLANCO", "VOTOS NULOS", "VOTOS NO MARCADOS"]
# Filtramos el dataframe para mantener solo las filas donde CANDIDATO NO está en la lista de valores a eliminar
votaciones_nacional_2014 = votaciones_nacional_2014[~votaciones_nacional_2014['CANDIDATO'].isin(valores_a_eliminar)]

In [6]:
votaciones_nacional_2014.head()

,CANDIDATO,VOTOS
0,LOPEZ,9
1,RAMIREZ,15
2,SANTOS,37
3,PEÑALOSA,2
4,ZULUAGA,42


In [7]:
# Añadir columna de ideología para el conjunto de 2014
# Crear diccionario de mapeo de candidatos a ideología para 2014
ideologia_2014 = {
    'LOPEZ': 'Izquierda',
    'RAMIREZ': 'Centro derecha',
    'SANTOS': 'Centro izquierda',
    'PEÑALOSA': 'Centro izquierda',
    'ZULUAGA': 'Derecha'
}

# Función para asignar ideología basada en el apellido del candidato
def asignar_ideologia_2014(candidato):
    for apellido, ideologia in ideologia_2014.items():
        if apellido in candidato:
            return ideologia
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de ideología
votaciones_nacional_2014['IDEOLOGIA'] = votaciones_nacional_2014['CANDIDATO'].apply(asignar_ideologia_2014)

# Verificar los resultados antes de eliminar la columna PARTIDO
print("Resultados después de añadir ideología (2014):")
print(votaciones_nacional_2014[['CANDIDATO', 'IDEOLOGIA']].drop_duplicates().sort_values('CANDIDATO'))



# Verificar la estructura del DataFrame después de eliminar la columna
print("\nEstructura después de eliminar la columna PARTIDO:")
print(votaciones_nacional_2014.columns.tolist())
print(votaciones_nacional_2014.head())

Resultados después de añadir ideología (2014):
  CANDIDATO         IDEOLOGIA
0     LOPEZ         Izquierda
3  PEÑALOSA  Centro izquierda
1   RAMIREZ    Centro derecha
2    SANTOS  Centro izquierda
4   ZULUAGA           Derecha

Estructura después de eliminar la columna PARTIDO:
['CANDIDATO', 'VOTOS', 'IDEOLOGIA']
  CANDIDATO  VOTOS         IDEOLOGIA
0     LOPEZ      9         Izquierda
1   RAMIREZ     15    Centro derecha
2    SANTOS     37  Centro izquierda
3  PEÑALOSA      2  Centro izquierda
4   ZULUAGA     42           Derecha


In [8]:
# Crear DataFrame de resultado para 2014
# Paso 1: Agrupar votos por ideología
votos_por_ideologia_2014 = votaciones_nacional_2014.groupby('IDEOLOGIA')['VOTOS'].sum().reset_index()

# Paso 2: Calcular el total de votos
total_votos_2014 = votaciones_nacional_2014['VOTOS'].sum()

# Paso 3: Calcular el porcentaje por ideología
votos_por_ideologia_2014['resultado'] = (votos_por_ideologia_2014['VOTOS'] / total_votos_2014 * 100).round(2)

# Paso 4: Crear el DataFrame final con solo ideología y resultado
resultado_2014 = votos_por_ideologia_2014[['IDEOLOGIA', 'resultado']]

# Mostrar el resultado
print("Resultado 2014 - Porcentaje de votos por ideología:")
print(resultado_2014)
print(f"\nTotal de votos analizados en 2014: {total_votos_2014}")

Resultado 2014 - Porcentaje de votos por ideología:
          IDEOLOGIA  resultado
0    Centro derecha      16.53
1  Centro izquierda      36.17
2           Derecha      31.11
3         Izquierda      16.20

Total de votos analizados en 2014: 12078927


In [9]:
# Añadir columna de Google trends en porcentaje semanal y mensual para el conjunto de 2014
# Crear diccionario de mapeo de candidatos a ideología para 2014
google_mensual_2014 = {
    'Izquierda': 27.46,
    'Centro derecha': 14.60,
    'Centro izquierda': (18.88 + 14.85),
    'Derecha': 24.19
}

# Función para asignar Google trends en porcentaje basada en el apellido del candidato
def asignar_google_mensual_2014(ideologia):
    for sector, google in google_mensual_2014.items():
        if sector in ideologia:
            return google
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de Google trends en porcentaje
resultado_2014['GOOGLE_MENSUAL'] = resultado_2014['IDEOLOGIA'].apply(asignar_google_mensual_2014)

# Verificar los resultados antes de eliminar la columna PARTIDO
print("Resultados después de añadir Google mensual (2014):")
print(resultado_2014[['IDEOLOGIA', 'GOOGLE_MENSUAL']].drop_duplicates().sort_values('IDEOLOGIA'))


print(resultado_2014.columns.tolist())
print(resultado_2014.head())

Resultados después de añadir Google mensual (2014):
          IDEOLOGIA  GOOGLE_MENSUAL
0    Centro derecha           14.60
1  Centro izquierda           33.73
2           Derecha           24.19
3         Izquierda           27.46
['IDEOLOGIA', 'resultado', 'GOOGLE_MENSUAL']
          IDEOLOGIA  resultado  GOOGLE_MENSUAL
0    Centro derecha      16.53           14.60
1  Centro izquierda      36.17           33.73
2           Derecha      31.11           24.19
3         Izquierda      16.20           27.46


In [10]:
# Añadir columna de Google trends en porcentaje semanal y mensual para el conjunto de 2014
# Crear diccionario de mapeo de candidatos a ideología para 2014
google_semanal_2014 = {
    'Izquierda': 29.68,
    'Centro derecha': 15.65,
    'Centro izquierda': (13.25 + 14.28),
    'Derecha': 27.11
}

# Función para asignar Google trends en porcentaje basada en el apellido del candidato
def asignar_google_semanal_2014(ideologia):
    for sector, google in google_semanal_2014.items():
        if sector in ideologia:
            return google
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de Google trends en porcentaje
resultado_2014['GOOGLE_SEMANAL'] = resultado_2014['IDEOLOGIA'].apply(asignar_google_semanal_2014)

# Verificar los resultados antes de eliminar la columna PARTIDO
print("Resultados después de añadir Google mensual (2014):")
print(resultado_2014[['IDEOLOGIA', 'GOOGLE_SEMANAL']].drop_duplicates().sort_values('IDEOLOGIA'))


print(resultado_2014.columns.tolist())
print(resultado_2014.head())

Resultados después de añadir Google mensual (2014):
          IDEOLOGIA  GOOGLE_SEMANAL
0    Centro derecha           15.65
1  Centro izquierda           27.53
2           Derecha           27.11
3         Izquierda           29.68
['IDEOLOGIA', 'resultado', 'GOOGLE_MENSUAL', 'GOOGLE_SEMANAL']
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL
0    Centro derecha      16.53           14.60           15.65
1  Centro izquierda      36.17           33.73           27.53
2           Derecha      31.11           24.19           27.11
3         Izquierda      16.20           27.46           29.68


In [11]:
# Incluiremos los diferentes valores de las encuestas nacionales en 2014
# Crear diccionarios con los valores de las encuestas

# Para encuesta 1 (primera imagen)
encuesta1_valores = {
    'Izquierda': 10.1,
    'Centro derecha': 9.7,
    'Centro izquierda': (28.5 + 9.4),
    'Derecha': 29.5
}

# Para encuesta 2 (Gallup)
encuesta2_valores = {
    'Izquierda': 10.9,
    'Centro derecha': 14.4,
    'Centro izquierda': (29.0 + 10.6),
    'Derecha': 29.3
}

# Para encuesta 3 (Datexto)
encuesta3_valores = {
    'Izquierda': 9.7,
    'Centro derecha': 9.4,
    'Centro izquierda': (29.7 + 9.7),
    'Derecha': 25.6
}

# Crear los nuevos dataframes
def crear_df_encuesta(resultado_df, valores_encuesta, nombre_encuesta):
    # Copiar el dataframe original
    nuevo_df = resultado_df.copy()
    
    # Añadir la columna de encuesta usando los valores por ideología
    nuevo_df['encuesta'] = nuevo_df['IDEOLOGIA'].map(valores_encuesta)
    
    # Añadir columna para identificar la encuesta
    nuevo_df['nombre_encuesta'] = nombre_encuesta
    
    return nuevo_df

# Crear los nuevos dataframes
df_2014_encuesta1 = crear_df_encuesta(resultado_2014, encuesta1_valores, "Encuesta 1")
df_2014_encuesta2 = crear_df_encuesta(resultado_2014, encuesta2_valores, "Gallup")
df_2014_encuesta3 = crear_df_encuesta(resultado_2014, encuesta3_valores, "Datexto")

# Mostrar los dataframes resultantes
print("Dataframe 2014 Encuesta 1:")
print(df_2014_encuesta1)
print("\nDataframe 2014 Encuesta 2 (Gallup):")
print(df_2014_encuesta2)
print("\nDataframe 2014 Encuesta 3 (Datexto):")
print(df_2014_encuesta3)

Dataframe 2014 Encuesta 1:
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL  encuesta  \
0    Centro derecha      16.53           14.60           15.65       9.7   
1  Centro izquierda      36.17           33.73           27.53      37.9   
2           Derecha      31.11           24.19           27.11      29.5   
3         Izquierda      16.20           27.46           29.68      10.1   

  nombre_encuesta  
0      Encuesta 1  
1      Encuesta 1  
2      Encuesta 1  
3      Encuesta 1  

Dataframe 2014 Encuesta 2 (Gallup):
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL  encuesta  \
0    Centro derecha      16.53           14.60           15.65      14.4   
1  Centro izquierda      36.17           33.73           27.53      39.6   
2           Derecha      31.11           24.19           27.11      29.3   
3         Izquierda      16.20           27.46           29.68      10.9   

  nombre_encuesta  
0          Gallup  
1          Gallup  
2          Gal

# Proceso de limpieza de elecciones 2018


In [14]:

def read_votaciones_2018_1v():
    votaciones_2018 = pd.read_excel('./in/MMV_NACIONAL_PRESIDENTE_2018_1v.xlsx')
    return votaciones_2018

# Get the data
votaciones_2018 = read_votaciones_2018_1v()

In [24]:
votaciones_2018.head()

,Unnamed: 0,DEP,DEPARTAMENTO,MUN,MUNICIPIO,Capital,ZONA,PSTO,MESA,COR,CIR,PAR,PARTIDO,CODCAN,CANDIDATO,VOTOS
0,11,1,ANTIOQUIA,1,MEDELLIN,Capital,1,1,1,1,0,12,PARTIDO CENTRO DEMOCRATICO,3,IVAN DUQUE,130
1,11,1,ANTIOQUIA,1,MEDELLIN,Capital,1,1,2,1,0,12,PARTIDO CENTRO DEMOCRATICO,3,IVAN DUQUE,37
2,11,1,ANTIOQUIA,1,MEDELLIN,Capital,1,1,3,1,0,12,PARTIDO CENTRO DEMOCRATICO,3,IVAN DUQUE,38
3,11,1,ANTIOQUIA,1,MEDELLIN,Capital,1,1,4,1,0,12,PARTIDO CENTRO DEMOCRATICO,3,IVAN DUQUE,71
4,11,1,ANTIOQUIA,1,MEDELLIN,Capital,1,1,5,1,0,12,PARTIDO CENTRO DEMOCRATICO,3,IVAN DUQUE,109


In [29]:
votaciones_nacional_2018 = votaciones_2018.drop(['Unnamed: 0', 'DEP', 'DEPARTAMENTO', 'MUN', 'MUNICIPIO',	'Capital', 'ZONA', 'PSTO', 'MESA','COR', 'CIR', 'PAR', 'PARTIDO  ' ,'CODCAN'], axis = 1)

votaciones_nacional_2018.head()

,CANDIDATO,VOTOS
0,IVAN DUQUE,130
1,IVAN DUQUE,37
2,IVAN DUQUE,38
3,IVAN DUQUE,71
4,IVAN DUQUE,109


In [27]:
votaciones_nacional_2018.columns

Index(['PARTIDO  ', 'CANDIDATO', 'VOTOS'], dtype='object')

In [30]:
# Eliminar filas donde CANDIDATO sea alguno de estos valores
valores_a_eliminar = ["VOTOS EN BLANCO", "VOTOS NULOS", "VOTOS NO MARCADOS"]
# Filtramos el dataframe para mantener solo las filas donde CANDIDATO NO está en la lista de valores a eliminar
votaciones_nacional_2018 = votaciones_nacional_2018[~votaciones_nacional_2018['CANDIDATO'].isin(valores_a_eliminar)]

In [31]:
votaciones_nacional_2018.head()

,CANDIDATO,VOTOS
0,IVAN DUQUE,130
1,IVAN DUQUE,37
2,IVAN DUQUE,38
3,IVAN DUQUE,71
4,IVAN DUQUE,109


In [39]:
# Añadir columna de ideología para el conjunto de 2018
# Crear diccionario de mapeo de candidatos a ideología para 2018
ideologia_2018 = {
    'IVAN DUQUE': 'Derecha',
    'GUSTAVO PETRO': 'Izquierda',
    'GERMAN VARGAS': 'Centro derecha',
    'HUMBERTO DE LA CALLE': 'Centro izquierda',
    'SERGIO FAJARDO': 'Centro izquierda'
}

# Función para asignar ideología basada en el nombre del candidato
def asignar_ideologia_2018(candidato):
    for nombre, ideologia in ideologia_2018.items():
        if nombre in candidato:
            return ideologia
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de ideología
votaciones_nacional_2018['IDEOLOGIA'] = votaciones_nacional_2018['CANDIDATO'].apply(asignar_ideologia_2018)



# Verificar cuántos registros tienen ideología no identificada
print("\nCandidatos con ideología no identificada:")
no_identificados = votaciones_nacional_2018[votaciones_nacional_2018['IDEOLOGIA'] == 'No identificada']
print(no_identificados['CANDIDATO'].drop_duplicates().tolist())
print(f"Total de filas con ideología no identificada: {len(no_identificados)}")

# Filtrar para mantener solo los registros con ideología identificada
bar_2018_con_ideologia = votaciones_nacional_2018[votaciones_nacional_2018['IDEOLOGIA'] != 'No identificada']
print(f"\nFilas eliminadas: {len(votaciones_nacional_2018) - len(bar_2018_con_ideologia)}")
print(f"Filas restantes: {len(votaciones_nacional_2018)}")

# Actualizar el DataFrame
votaciones_nacional_2018 = bar_2018_con_ideologia

# Verificar la estructura final del DataFrame
print("\nEstructura final del DataFrame:")
print(votaciones_nacional_2018.columns.tolist())
print(votaciones_nacional_2018[['CANDIDATO', 'IDEOLOGIA']].drop_duplicates().sort_values('CANDIDATO'))


Candidatos con ideología no identificada:
[]
Total de filas con ideología no identificada: 0

Filas eliminadas: 0
Filas restantes: 467745

Estructura final del DataFrame:
['CANDIDATO', 'VOTOS', 'IDEOLOGIA']
                   CANDIDATO         IDEOLOGIA
342127  GERMAN VARGAS LLERAS    Centro derecha
458231         GUSTAVO PETRO         Izquierda
162743  HUMBERTO DE LA CALLE  Centro izquierda
0                 IVAN DUQUE           Derecha
246808        SERGIO FAJARDO  Centro izquierda


In [40]:
# Crear DataFrame de resultado para 2018
# Paso 1: Agrupar votos por ideología
votos_por_ideologia_2018 = votaciones_nacional_2018.groupby('IDEOLOGIA')['VOTOS'].sum().reset_index()

# Paso 2: Calcular el total de votos
total_votos_2018 = votaciones_nacional_2018['VOTOS'].sum()

# Paso 3: Calcular el porcentaje por ideología
votos_por_ideologia_2018['resultado'] = (votos_por_ideologia_2018['VOTOS'] / total_votos_2018 * 100).round(2)

# Paso 4: Crear el DataFrame final con solo ideología y resultado
resultado_2018 = votos_por_ideologia_2018[['IDEOLOGIA', 'resultado']]

# Mostrar el resultado
print("Resultado 2018 - Porcentaje de votos por ideología:")
print(resultado_2018)
print(f"\nTotal de votos analizados en 2018: {total_votos_2018}")

Resultado 2018 - Porcentaje de votos por ideología:
          IDEOLOGIA  resultado
0    Centro derecha       7.48
1  Centro izquierda      26.47
2           Derecha      40.34
3         Izquierda      25.71

Total de votos analizados en 2018: 18883385


In [41]:
# Añadir columna de Google trends en porcentaje semanal y mensual para el conjunto de 2018
# Crear diccionario de mapeo de candidatos a ideología para 2018
google_mensual_2018 = {
    'Izquierda': 44.17,
    'Centro derecha': 9.39,
    'Centro izquierda': (4.7 + 16.47),
    'Derecha': 25.18
}

# Función para asignar Google trends en porcentaje basada en el apellido del candidato
def asignar_google_mensual_2018(ideologia):
    for sector, google in google_mensual_2018.items():
        if sector in ideologia:
            return google
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de Google trends en porcentaje
resultado_2018['GOOGLE_MENSUAL'] = resultado_2018['IDEOLOGIA'].apply(asignar_google_mensual_2018)

# Verificar los resultados antes de eliminar la columna PARTIDO
print("Resultados después de añadir Google mensual (2018):")
print(resultado_2018[['IDEOLOGIA', 'GOOGLE_MENSUAL']].drop_duplicates().sort_values('IDEOLOGIA'))


print(resultado_2018.columns.tolist())
print(resultado_2018.head())

Resultados después de añadir Google mensual (2018):
          IDEOLOGIA  GOOGLE_MENSUAL
0    Centro derecha            9.39
1  Centro izquierda           21.17
2           Derecha           25.18
3         Izquierda           44.17
['IDEOLOGIA', 'resultado', 'GOOGLE_MENSUAL']
          IDEOLOGIA  resultado  GOOGLE_MENSUAL
0    Centro derecha       7.48            9.39
1  Centro izquierda      26.47           21.17
2           Derecha      40.34           25.18
3         Izquierda      25.71           44.17


In [42]:
# Añadir columna de Google trends en porcentaje semanal y mensual para el conjunto de 2018
# Crear diccionario de mapeo de candidatos a ideología para 2018
google_semanal_2018 = {
    'Izquierda': 40.21,
    'Centro derecha': 9.9,
    'Centro izquierda': (4.7 + 21.46),
    'Derecha': 23.61
}

# Función para asignar Google trends en porcentaje basada en el apellido del candidato
def asignar_google_semanal_2018(ideologia):
    for sector, google in google_semanal_2018.items():
        if sector in ideologia:
            return google
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de Google trends en porcentaje
resultado_2018['GOOGLE_SEMANAL'] = resultado_2018['IDEOLOGIA'].apply(asignar_google_semanal_2018)

# Verificar los resultados antes de eliminar la columna PARTIDO
print("Resultados después de añadir Google mensual (2018):")
print(resultado_2018[['IDEOLOGIA', 'GOOGLE_SEMANAL']].drop_duplicates().sort_values('IDEOLOGIA'))


print(resultado_2018.columns.tolist())
print(resultado_2018.head())

Resultados después de añadir Google mensual (2018):
          IDEOLOGIA  GOOGLE_SEMANAL
0    Centro derecha            9.90
1  Centro izquierda           26.16
2           Derecha           23.61
3         Izquierda           40.21
['IDEOLOGIA', 'resultado', 'GOOGLE_MENSUAL', 'GOOGLE_SEMANAL']
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL
0    Centro derecha       7.48            9.39            9.90
1  Centro izquierda      26.47           21.17           26.16
2           Derecha      40.34           25.18           23.61
3         Izquierda      25.71           44.17           40.21


In [44]:
# Incluiremos los diferentes valores de las encuestas nacionales en 2018
# Crear diccionarios con los valores de las encuestas

# Para encuesta 1 (CNC)
encuesta1_valores = {
    'Izquierda': 27,
    'Centro derecha': 10,
    'Centro izquierda': (18 + 4),
    'Derecha': 36
}

# Para encuesta 2 (Guarumo)
encuesta2_valores = {
    'Izquierda': 24.4,
    'Centro derecha': 11,
    'Centro izquierda': (16 + 2.3),
    'Derecha': 37.6
}

# Para encuesta 3 (Invamer)
encuesta3_valores = {
    'Izquierda': 31,
    'Centro derecha': 7.9,
    'Centro izquierda': (2.5 + 13.3),
    'Derecha': 41.3
}

# Crear los nuevos dataframes
def crear_df_encuesta(resultado_df, valores_encuesta, nombre_encuesta):
    # Copiar el dataframe original
    nuevo_df = resultado_df.copy()
    
    # Añadir la columna de encuesta usando los valores por ideología
    nuevo_df['encuesta'] = nuevo_df['IDEOLOGIA'].map(valores_encuesta)
    
    # Añadir columna para identificar la encuesta
    nuevo_df['nombre_encuesta'] = nombre_encuesta
    
    return nuevo_df

# Crear los nuevos dataframes
df_2018_encuesta1 = crear_df_encuesta(resultado_2018, encuesta1_valores, "CNC")
df_2018_encuesta2 = crear_df_encuesta(resultado_2018, encuesta2_valores, "Guarumo")
df_2018_encuesta3 = crear_df_encuesta(resultado_2018, encuesta3_valores, "Invamer")

# Mostrar los dataframes resultantes
print("Dataframe 2018 Encuesta 1 (CNC):")
print(df_2018_encuesta1)
print("\nDataframe 2018 Encuesta 2 (Guarumo):")
print(df_2018_encuesta2)
print("\nDataframe 2018 Encuesta 3 (Invamer):")
print(df_2018_encuesta3)

Dataframe 2018 Encuesta 1 (CNC):
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL  encuesta  \
0    Centro derecha       7.48            9.39            9.90        10   
1  Centro izquierda      26.47           21.17           26.16        22   
2           Derecha      40.34           25.18           23.61        36   
3         Izquierda      25.71           44.17           40.21        27   

  nombre_encuesta  
0             CNC  
1             CNC  
2             CNC  
3             CNC  

Dataframe 2018 Encuesta 2 (Guarumo):
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL  encuesta  \
0    Centro derecha       7.48            9.39            9.90      11.0   
1  Centro izquierda      26.47           21.17           26.16      18.3   
2           Derecha      40.34           25.18           23.61      37.6   
3         Izquierda      25.71           44.17           40.21      24.4   

  nombre_encuesta  
0         Guarumo  
1         Guarumo  
2      

# Proceso de limpieza de elecciones 2022


In [45]:

def read_votaciones_2022_1v():
    votaciones_2022 = pd.read_excel('./in/MMV_NACIONAL_PRESIDENTE_2022_1v.xlsx')
    return votaciones_2022

# Get the data
votaciones_2022 = read_votaciones_2022_1v()

In [46]:
votaciones_2022.head()

,Unnamed: 0,DEP,DEPNOMBRE,MUN,MUNNOMBRE,ZONA,PUESTO,PUESNOMBRE,MESA,CORCODIGO,CORNOMBRE,CIR,PAR,PARNOMBRE,CAN,CANNOMBRE,VOTOS
0,88120,88,CONSULADOS,120,ALEMANIA,5,2,BERLIN CONSULADO,4,1,PRESIDENTE,0,1076,LIGA DE GOBERNANTES ANTICORRUPCION ...,1,RODOLFO HERNÁNDEZ,3
1,88120,88,CONSULADOS,120,ALEMANIA,5,2,BERLIN CONSULADO,4,1,PRESIDENTE,0,996,VOTOS EN BLANCO,996,VOTOS EN BLANCO .,2
2,88120,88,CONSULADOS,120,ALEMANIA,5,2,BERLIN CONSULADO,4,1,PRESIDENTE,0,1235,COALICION PACTO HISTÓRICO ...,6,GUSTAVO PETRO,53
3,88120,88,CONSULADOS,120,ALEMANIA,5,2,BERLIN CONSULADO,4,1,PRESIDENTE,0,1233,COALICION CENTRO ESPERANZA ...,4,SERGIO FAJARDO,13
4,88120,88,CONSULADOS,120,ALEMANIA,5,2,BERLIN CONSULADO,4,1,PRESIDENTE,0,1234,COALICION EQUIPO POR COLOMBIA ...,3,FEDERICO GUTIÉRREZ,4


In [49]:
votaciones_nacional_2022 = votaciones_2022.drop(['Unnamed: 0','DEP',	'DEPNOMBRE',	'MUN',	'MUNNOMBRE', 'ZONA', 'PUESTO', 'PUESNOMBRE', 'MESA', 'CORCODIGO',	'CORNOMBRE',	'CIR','PAR',	'PARNOMBRE','CAN'], axis = 1)

In [50]:
votaciones_nacional_2022.head()

,CANNOMBRE,VOTOS
0,RODOLFO HERNÁNDEZ,3
1,VOTOS EN BLANCO .,2
2,GUSTAVO PETRO,53
3,SERGIO FAJARDO,13
4,FEDERICO GUTIÉRREZ,4


In [51]:
# Eliminar filas donde CANDIDATO sea alguno de estos valores
valores_a_eliminar = ["VOTOS EN BLANCO .", "VOTOS NULOS", "VOTOS NO MARCADOS"]
# Filtramos el dataframe para mantener solo las filas donde CANDIDATO NO está en la lista de valores a eliminar
votaciones_nacional_2022 = votaciones_nacional_2022[~votaciones_nacional_2022['CANNOMBRE'].isin(valores_a_eliminar)]

In [52]:
votaciones_nacional_2022.head()

,CANNOMBRE,VOTOS
0,RODOLFO HERNÁNDEZ,3
2,GUSTAVO PETRO,53
3,SERGIO FAJARDO,13
4,FEDERICO GUTIÉRREZ,4
5,JOHN MILTON RODRÍGUEZ,1


In [53]:
# Añadir columna de ideología para el conjunto de 2022
# Crear diccionario de mapeo de candidatos a ideología para 2022
ideologia_2022 = {
    'FEDERICO GUTIÉRREZ': 'Derecha',
    'GUSTAVO PETRO': 'Izquierda',
    'RODOLFO HERNÁNDEZ': 'Centro derecha',
    'SERGIO FAJARDO': 'Centro izquierda'
}

# Función para asignar ideología basada en el nombre del candidato
def asignar_ideologia_2022(candidato):
    for nombre, ideologia in ideologia_2022.items():
        if nombre in candidato:
            return ideologia
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de ideología
votaciones_nacional_2022['IDEOLOGIA'] = votaciones_nacional_2022['CANNOMBRE'].apply(asignar_ideologia_2022)


# Verificar cuántos registros tienen ideología no identificada
print("\nCandidatos con ideología no identificada:")
no_identificados = votaciones_nacional_2022[votaciones_nacional_2022['IDEOLOGIA'] == 'No identificada']
print(no_identificados['CANNOMBRE'].drop_duplicates().tolist())
print(f"Total de filas con ideología no identificada: {len(no_identificados)}")

# Filtrar para mantener solo los registros con ideología identificada
bar_2022_con_ideologia = votaciones_nacional_2022[votaciones_nacional_2022['IDEOLOGIA'] != 'No identificada']
print(f"\nFilas eliminadas: {len(votaciones_nacional_2022) - len(bar_2022_con_ideologia)}")
print(f"Filas restantes: {len(bar_2022_con_ideologia)}")

# Actualizar el DataFrame
votaciones_nacional_2022 = bar_2022_con_ideologia

# Verificar la estructura final del DataFrame
print("\nEstructura final del DataFrame:")
print(votaciones_nacional_2022.columns.tolist())
print(votaciones_nacional_2022[['CANNOMBRE', 'IDEOLOGIA']].drop_duplicates().sort_values('CANNOMBRE'))


Candidatos con ideología no identificada:
['JOHN MILTON RODRÍGUEZ', 'INGRID BETANCOURT', 'LUIS PÉREZ', 'VOTOS NULOS .', 'ENRIQUE GÓMEZ MARTÍNEZ', 'VOTOS NO MARCADOS .']
Total de filas con ideología no identificada: 233505

Filas eliminadas: 233505
Filas restantes: 402564

Estructura final del DataFrame:
['CANNOMBRE', 'VOTOS', 'IDEOLOGIA']
            CANNOMBRE         IDEOLOGIA
4  FEDERICO GUTIÉRREZ           Derecha
2       GUSTAVO PETRO         Izquierda
0   RODOLFO HERNÁNDEZ    Centro derecha
3      SERGIO FAJARDO  Centro izquierda


In [55]:
votaciones_nacional_2022.head()

,CANNOMBRE,VOTOS,IDEOLOGIA
0,RODOLFO HERNÁNDEZ,3,Centro derecha
2,GUSTAVO PETRO,53,Izquierda
3,SERGIO FAJARDO,13,Centro izquierda
4,FEDERICO GUTIÉRREZ,4,Derecha
6,RODOLFO HERNÁNDEZ,20,Centro derecha


In [56]:
# Crear DataFrame de resultado para 2022
# Paso 1: Agrupar votos por ideología
votos_por_ideologia_2022 = votaciones_nacional_2022.groupby('IDEOLOGIA')['VOTOS'].sum().reset_index()

# Paso 2: Calcular el total de votos
total_votos_2022 = votaciones_nacional_2022['VOTOS'].sum()

# Paso 3: Calcular el porcentaje por ideología
votos_por_ideologia_2022['resultado'] = (votos_por_ideologia_2022['VOTOS'] / total_votos_2022 * 100).round(2)

# Paso 4: Crear el DataFrame final con solo ideología y resultado
resultado_2022 = votos_por_ideologia_2022[['IDEOLOGIA', 'resultado']]

# Mostrar el resultado
print("Resultado 2022 - Porcentaje de votos por ideología:")
print(resultado_2022)
print(f"\nTotal de votos analizados en 2022: {total_votos_2022}")

Resultado 2022 - Porcentaje de votos por ideología:
          IDEOLOGIA  resultado
0    Centro derecha      29.15
1  Centro izquierda       4.33
2           Derecha      24.77
3         Izquierda      41.75

Total de votos analizados en 2022: 20462368


In [57]:
# Añadir columna de Google trends en porcentaje semanal y mensual para el conjunto de 2022
# Crear diccionario de mapeo de candidatos a ideología para 2022
google_mensual_2022 = {
    'Izquierda': 49.28,
    'Centro derecha': 32.9,
    'Centro izquierda': 8.1,
    'Derecha': 9.6
}

# Función para asignar Google trends en porcentaje basada en el apellido del candidato
def asignar_google_mensual_2022(ideologia):
    for sector, google in google_mensual_2022.items():
        if sector in ideologia:
            return google
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de Google trends en porcentaje
resultado_2022['GOOGLE_MENSUAL'] = resultado_2022['IDEOLOGIA'].apply(asignar_google_mensual_2022)

# Verificar los resultados antes de eliminar la columna PARTIDO
print("Resultados después de añadir Google mensual (2022):")
print(resultado_2022[['IDEOLOGIA', 'GOOGLE_MENSUAL']].drop_duplicates().sort_values('IDEOLOGIA'))


print(resultado_2022.columns.tolist())
print(resultado_2022.head())

Resultados después de añadir Google mensual (2022):
          IDEOLOGIA  GOOGLE_MENSUAL
0    Centro derecha           32.90
1  Centro izquierda            8.10
2           Derecha            9.60
3         Izquierda           49.28
['IDEOLOGIA', 'resultado', 'GOOGLE_MENSUAL']
          IDEOLOGIA  resultado  GOOGLE_MENSUAL
0    Centro derecha      29.15           32.90
1  Centro izquierda       4.33            8.10
2           Derecha      24.77            9.60
3         Izquierda      41.75           49.28


In [58]:
# Añadir columna de Google trends en porcentaje semanal y mensual para el conjunto de 2022
# Crear diccionario de mapeo de candidatos a ideología para 2022
google_semanal_2022 = {
    'Izquierda': 44.11,
    'Centro derecha': 39.13,
    'Centro izquierda': 8.9,
    'Derecha': 7.8
}

# Función para asignar Google trends en porcentaje basada en el apellido del candidato
def asignar_google_semanal_2022(ideologia):
    for sector, google in google_semanal_2022.items():
        if sector in ideologia:
            return google
    return 'No identificada'  # Para candidatos que no estén en la lista

# Aplicar la función para añadir la columna de Google trends en porcentaje
resultado_2022['GOOGLE_SEMANAL'] = resultado_2022['IDEOLOGIA'].apply(asignar_google_semanal_2022)

# Verificar los resultados antes de eliminar la columna PARTIDO
print("Resultados después de añadir Google mensual (2022):")
print(resultado_2022[['IDEOLOGIA', 'GOOGLE_SEMANAL']].drop_duplicates().sort_values('IDEOLOGIA'))


print(resultado_2022.columns.tolist())
print(resultado_2022.head())

Resultados después de añadir Google mensual (2022):
          IDEOLOGIA  GOOGLE_SEMANAL
0    Centro derecha           39.13
1  Centro izquierda            8.90
2           Derecha            7.80
3         Izquierda           44.11
['IDEOLOGIA', 'resultado', 'GOOGLE_MENSUAL', 'GOOGLE_SEMANAL']
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL
0    Centro derecha      29.15           32.90           39.13
1  Centro izquierda       4.33            8.10            8.90
2           Derecha      24.77            9.60            7.80
3         Izquierda      41.75           49.28           44.11


In [59]:
# Incluiremos los diferentes valores de las encuestas nacionales en 2018
# Crear diccionarios con los valores de las encuestas

# Para encuesta 1 (Guarumo)
encuesta1_valores = {
    'Izquierda': 37.9,
    'Centro derecha': 20.3,
    'Centro izquierda': 4.3,
    'Derecha': 30.8
}

# Para encuesta 2 (CNC)
encuesta2_valores = {
    'Izquierda': 41,
    'Centro derecha': 21.9,
    'Centro izquierda': 4.5,
    'Derecha': 23.9
}

# Para encuesta 3 (Invamer)
encuesta3_valores = {
    'Izquierda': 40.6,
    'Centro derecha': 20.9,
    'Centro izquierda': 5.1,
    'Derecha': 27.1
}

# Crear los nuevos dataframes
def crear_df_encuesta(resultado_df, valores_encuesta, nombre_encuesta):
    # Copiar el dataframe original
    nuevo_df = resultado_df.copy()
    
    # Añadir la columna de encuesta usando los valores por ideología
    nuevo_df['encuesta'] = nuevo_df['IDEOLOGIA'].map(valores_encuesta)
    
    # Añadir columna para identificar la encuesta
    nuevo_df['nombre_encuesta'] = nombre_encuesta
    
    return nuevo_df

# Crear los nuevos dataframes
df_2022_encuesta1 = crear_df_encuesta(resultado_2022, encuesta1_valores, "Guarumo")
df_2022_encuesta2 = crear_df_encuesta(resultado_2022, encuesta2_valores, "CNC")
df_2022_encuesta3 = crear_df_encuesta(resultado_2022, encuesta3_valores, "Invamer")

# Mostrar los dataframes resultantes
print("Dataframe 2022 Encuesta 1 (Guarumo):")
print(df_2022_encuesta1)
print("\nDataframe 2022 Encuesta 2 (CNC):")
print(df_2022_encuesta2)
print("\nDataframe 2022 Encuesta 3 (Invamer):")
print(df_2022_encuesta3)

Dataframe 2022 Encuesta 1 (Guarumo):
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL  encuesta  \
0    Centro derecha      29.15           32.90           39.13      20.3   
1  Centro izquierda       4.33            8.10            8.90       4.3   
2           Derecha      24.77            9.60            7.80      30.8   
3         Izquierda      41.75           49.28           44.11      37.9   

  nombre_encuesta  
0         Guarumo  
1         Guarumo  
2         Guarumo  
3         Guarumo  

Dataframe 2022 Encuesta 2 (CNC):
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL  encuesta  \
0    Centro derecha      29.15           32.90           39.13      21.9   
1  Centro izquierda       4.33            8.10            8.90       4.5   
2           Derecha      24.77            9.60            7.80      23.9   
3         Izquierda      41.75           49.28           44.11      41.0   

  nombre_encuesta  
0             CNC  
1             CNC  
2      

In [60]:
#Vamos a escalar los valores importantes para que tengan mas peso al momento de creación del modelo
#Para esto le daremos más peso a la información más actualizada
# Dividiremos por 2 y por 3 los datos más antiguos para escalarlos, y asi no perder coherencia al momento de que se escalen las variables
df_2018_encuesta1['resultado'] = df_2018_encuesta1['resultado'] / 2
df_2018_encuesta1['encuesta'] = df_2018_encuesta1['encuesta'] / 2
df_2018_encuesta1['GOOGLE_MENSUAL'] = df_2018_encuesta1['GOOGLE_MENSUAL'] / 2
df_2018_encuesta1['GOOGLE_SEMANAL'] = df_2018_encuesta1['GOOGLE_SEMANAL'] / 2

df_2018_encuesta2['resultado'] = df_2018_encuesta2['resultado'] / 2
df_2018_encuesta2['encuesta'] = df_2018_encuesta2['encuesta'] / 2
df_2018_encuesta2['GOOGLE_MENSUAL'] = df_2018_encuesta2['GOOGLE_MENSUAL'] / 2
df_2018_encuesta2['GOOGLE_SEMANAL'] = df_2018_encuesta2['GOOGLE_SEMANAL'] / 2

df_2018_encuesta3['resultado'] = df_2018_encuesta3['resultado'] / 2
df_2018_encuesta3['encuesta'] = df_2018_encuesta3['encuesta'] / 2
df_2018_encuesta3['GOOGLE_MENSUAL'] = df_2018_encuesta3['GOOGLE_MENSUAL'] / 2
df_2018_encuesta3['GOOGLE_SEMANAL'] = df_2018_encuesta3['GOOGLE_SEMANAL'] / 2

# Verificar uno de los dataframes de 2018 después de la división
print("Dataframe 2018 Encuesta 1 después de dividir por 2:")
print(df_2018_encuesta1)

# DIvidir por 3 los valores de las columnas 'resultado' y 'encuesta' en los dataframes de 2014
df_2014_encuesta1['resultado'] = df_2014_encuesta1['resultado'] / 3
df_2014_encuesta1['encuesta'] = df_2014_encuesta1['encuesta'] / 3
df_2014_encuesta1['GOOGLE_MENSUAL'] = df_2014_encuesta1['GOOGLE_MENSUAL'] / 3
df_2014_encuesta1['GOOGLE_SEMANAL'] = df_2014_encuesta1['GOOGLE_SEMANAL'] / 3

df_2014_encuesta2['resultado'] = df_2014_encuesta2['resultado'] / 3
df_2014_encuesta2['encuesta'] = df_2014_encuesta2['encuesta'] / 3
df_2014_encuesta2['GOOGLE_MENSUAL'] = df_2014_encuesta2['GOOGLE_MENSUAL'] / 3
df_2014_encuesta2['GOOGLE_SEMANAL'] = df_2014_encuesta2['GOOGLE_SEMANAL'] / 3

df_2014_encuesta3['resultado'] = df_2014_encuesta3['resultado'] / 3
df_2014_encuesta3['encuesta'] = df_2014_encuesta3['encuesta'] / 3
df_2014_encuesta3['GOOGLE_MENSUAL'] = df_2014_encuesta3['GOOGLE_MENSUAL'] / 3
df_2014_encuesta3['GOOGLE_SEMANAL'] = df_2014_encuesta3['GOOGLE_SEMANAL'] / 3

# Verificar uno de los dataframes de 2014 después de la división
print("\nDataframe 2014 Encuesta 1 después de dividir por 3:")
print(df_2014_encuesta1)

Dataframe 2018 Encuesta 1 después de dividir por 2:
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL  encuesta  \
0    Centro derecha      3.740           4.695           4.950       5.0   
1  Centro izquierda     13.235          10.585          13.080      11.0   
2           Derecha     20.170          12.590          11.805      18.0   
3         Izquierda     12.855          22.085          20.105      13.5   

  nombre_encuesta  
0             CNC  
1             CNC  
2             CNC  
3             CNC  

Dataframe 2014 Encuesta 1 después de dividir por 3:
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL   encuesta  \
0    Centro derecha   5.510000        4.866667        5.216667   3.233333   
1  Centro izquierda  12.056667       11.243333        9.176667  12.633333   
2           Derecha  10.370000        8.063333        9.036667   9.833333   
3         Izquierda   5.400000        9.153333        9.893333   3.366667   

  nombre_encuesta  
0      E

In [63]:
# Primero, vamos a añadir una columna de 'año' a cada dataframe para identificar el año electoral
# Dataframes de 2014
df_2014_encuesta1['año'] = 2014
df_2014_encuesta2['año'] = 2014
df_2014_encuesta3['año'] = 2014

# Dataframes de 2018
df_2018_encuesta1['año'] = 2018
df_2018_encuesta2['año'] = 2018
df_2018_encuesta3['año'] = 2018

# Dataframes de 2022
df_2022_encuesta1['año'] = 2022
df_2022_encuesta2['año'] = 2022
df_2022_encuesta3['año'] = 2022

# Unir todos los dataframes
checkpoint_nacional = pd.concat([
    df_2014_encuesta1, df_2014_encuesta2, df_2014_encuesta3,
    df_2018_encuesta1, df_2018_encuesta2, df_2018_encuesta3,
    df_2022_encuesta1, df_2022_encuesta2, df_2022_encuesta3
], ignore_index=True)

# Mostrar el resultado
print("Primeras filas del dataframe unificado 'checkpoint_nacional':")
print(checkpoint_nacional.head())

print("\nInformación sobre el dataframe unificado:")
print(f"Número total de filas: {len(checkpoint_nacional)}")
print(f"Columnas: {checkpoint_nacional.columns.tolist()}")

# Verificar cuántas filas hay por año y encuesta
print("\nDistribución por año y encuesta:")
print(checkpoint_nacional.groupby(['año', 'nombre_encuesta']).size())

Primeras filas del dataframe unificado 'checkpoint_nacional':
          IDEOLOGIA  resultado  GOOGLE_MENSUAL  GOOGLE_SEMANAL   encuesta  \
0    Centro derecha   5.510000        4.866667        5.216667   3.233333   
1  Centro izquierda  12.056667       11.243333        9.176667  12.633333   
2           Derecha  10.370000        8.063333        9.036667   9.833333   
3         Izquierda   5.400000        9.153333        9.893333   3.366667   
4    Centro derecha   5.510000        4.866667        5.216667   4.800000   

  nombre_encuesta   año  
0      Encuesta 1  2014  
1      Encuesta 1  2014  
2      Encuesta 1  2014  
3      Encuesta 1  2014  
4          Gallup  2014  

Información sobre el dataframe unificado:
Número total de filas: 36
Columnas: ['IDEOLOGIA', 'resultado', 'GOOGLE_MENSUAL', 'GOOGLE_SEMANAL', 'encuesta', 'nombre_encuesta', 'año']

Distribución por año y encuesta:
año   nombre_encuesta
2014  Datexto            4
      Encuesta 1         4
      Gallup             4
20

In [64]:
# Guardar el dataframe en formato CSV en la carpeta "out"
checkpoint_nacional.to_csv('./out/checkpoint_nacional.csv', index=False)
print("El dataframe ha sido guardado exitosamente")

El dataframe ha sido guardado exitosamente
